# Task 1: Binary Representations

This section implements fundamental bitwise operations used in cryptographic functions.
We define:
- **rotl(x, n)**: Left-rotates a 32-bit integer `x` by `n` places.
- **rotr(x, n)**: Right-rotates a 32-bit integer `x` by `n` places.
- **ch(x, y, z)**: Chooses bits based on `x` (from `y` if `x` has 1s, from `z` otherwise).
- **maj(x, y, z)**: Computes the majority vote of the bits from `x, y, z`.

In [61]:
def rotl(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the left by n places.
    
    Args:
    - x (int): A 32-bit integer.
    - n (int): Number of positions to rotate.
    
    Returns:
    - int: The rotated integer.
    """
    n = n % 32  # Ensure n is within 0-31
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF  # Mask to 32 bits

def rotr(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the right by n places.
    """
    n = n % 32
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def ch(x, y, z):
    """
    Chooses bits from y where x has bits set to 1 and from z where x has bits set to 0.
    """
    return (x & y) ^ (~x & z)

def maj(x, y, z):
    """
    Takes a majority vote of the bits in x, y, and z.
    """
    return (x & y) ^ (x & z) ^ (y & z)


Testing the Bitwise Functions:
We use three test values:
- **x = 0xB3333333**
- **y = 0xCCCCCCCC**
- **z = 0xF0F0F0F0**

We will rotate `x`, apply `ch(x, y, z)`, and compute `maj(x, y, z)`.


In [62]:
# Test values
test_x = 0b10110011001100110011001100110011  # 0xB3333333
test_y = 0b11001100110011001100110011001100  # 0xCCCCCCCC
test_z = 0b11110000111100001111000011110000  # 0xF0F0F0F0

# Print the results
print(f"rotl({test_x:032b}, 4)  -> {rotl(test_x, 4):032b}")
print(f"rotr({test_x:032b}, 4)  -> {rotr(test_x, 4):032b}")
print(f"ch({test_x:032b}, {test_y:032b}, {test_z:032b}) -> {ch(test_x, test_y, test_z):032b}")
print(f"maj({test_x:032b}, {test_y:032b}, {test_z:032b}) -> {maj(test_x, test_y, test_z):032b}")


rotl(10110011001100110011001100110011, 4)  -> 00110011001100110011001100111011
rotr(10110011001100110011001100110011, 4)  -> 00111011001100110011001100110011
ch(10110011001100110011001100110011, 11001100110011001100110011001100, 11110000111100001111000011110000) -> 11000000110000001100000011000000
maj(10110011001100110011001100110011, 11001100110011001100110011001100, 11110000111100001111000011110000) -> 11110000111100001111000011110000


# Task 2: Hash Functions

This function computes a simple hash value for a string using the following logic:
1. Initialize `hashval = 0`.
2. Iterate over each character in the string.
3. Compute `hashval = ord(char) + 31 * hashval`.
4. Take the final value modulo 101.

This method ensures good hash distribution and minimizes collisions.


In [63]:
def hash_function(s: str) -> int:
    """
    Computes a simple hash value for a string using a rolling hash approach.
    
    Args:
    - s (str): The input string.
    
    Returns:
    - int: The hash value modulo 101.
    """
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval  # Hash function using 31
    return hashval % 101  # Modulo 101 to limit hash size


Testing the Hash Function:
We test the function with different strings to observe the hash values.


In [64]:
# Test cases
test_strings = ["hello", "world", "hashing", "kernighan", "ritchie"]

# Compute hashes
for s in test_strings:
    print(f"Hash of '{s}': {hash_function(s)}")


Hash of 'hello': 17
Hash of 'world': 34
Hash of 'hashing': 25
Hash of 'kernighan': 37
Hash of 'ritchie': 26


Use of 31 in Hash Functions:

The number 31 is commonly used in [hash values](https://www.geeksforgeeks.org/why-does-javas-hashcode-in-string-use-31-as-a-multiplier/)., such as Java's String.hashCode(), because it's a prime number, which helps in achieving a uniform distribution of hash values and reduces collisions. Additionally, multiplying by 31 can be optimized by the compiler to (x << 5) - x, improving performance. ​


Use of 101 as Modulus in Hash Functions:

Using a prime number like 101 as the [modulus in hash functions](https://www.designgurus.io/answers/detail/why-should-hash-functions-use-a-prime-number-modulus) ensures a more uniform distribution of hash values, minimizing clustering and collisions. This practice enhances the efficiency and effectiveness of the hash function.


# Task 3: SHA256 Padding

SHA256 requires messages to be padded to a multiple of 512 bits. 
This function:
1. Reads a file's contents.
2. Appends a `1` bit (`0x80` in hex).
3. Adds `0` bits until the total length is `56 mod 64`.
4. Appends the original message length as a big-endian 64-bit integer.

This padding ensures compatibility with SHA256 hashing.


In [65]:
import struct
import os

def calculate_sha256_padding(file_path):
    """
    Computes the SHA256 padding for a given file.
    
    Args:
    - file_path (str): The path to the input file.
    
    Prints:
    - The padding bytes in hexadecimal.
    """
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist.")
        return
    
    with open(file_path, 'rb') as f:
        data = f.read()

    print(f"Read {len(data)} bytes from {file_path}")

    original_length = len(data)
    original_bit_length = original_length * 8  # Convert bytes to bits

    # Append '1' bit (0x80 in hex)
    padding = b'\x80'

    # Compute required zero padding
    total_length = original_length + 1
    while (total_length + 8) % 64 != 0:
        padding += b'\x00'
        total_length += 1

    # Append original length in bits as a big-endian 64-bit integer
    padding += struct.pack('>Q', original_bit_length)

    print(f"Padding length: {len(padding)} bytes")
    print("Padding (Hex):", " ".join(f"{byte:02x}" for byte in padding))


Testing the SHA256 Padding Function:
We apply the padding function to 'test.txt`, which contains a short message.


In [66]:
# Run the padding function
calculate_sha256_padding("test.txt")

Read 3 bytes from test.txt
Padding length: 61 bytes
Padding (Hex): 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


# Task 4: Calculate the First 100 Prime Numbers Using Two Algorithms

## Overview
This notebook computes the first 100 prime numbers using:
1. **Trial Division Method** - A basic but slow approach.
2. **Sieve of Eratosthenes** - A more efficient method for generating primes.


## Implementing the [Trial Division Method](https://www.geeksforgeeks.org/trial-division-algorithm-for-prime-factorization/)

### How It Works:
- A number **n** is prime if it is **only divisible by 1 and itself**.
- To check if a number is prime:
  1. Start from `n = 2` and check divisibility up to `sqrt(n)`.
  2. If no divisor is found, it is prime.
  3. Continue finding primes until we reach **100 primes**.

#### **Efficiency**:
- **Time Complexity**: \( O(n \sqrt{n}) \) (not optimal for large numbers).


In [67]:
import math

def is_prime_trial_division(n):
    """Checks if a number is prime using trial division."""
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def first_n_primes_trial_division(n):
    """Finds the first n prime numbers using the trial division method."""
    primes = []
    num = 2  # Start checking from 2
    while len(primes) < n:
        if is_prime_trial_division(num):
            primes.append(num)
        num += 1
    return primes

# Compute the first 100 primes using Trial Division
primes_trial_division = first_n_primes_trial_division(100)

# Display the first 100 primes
primes_trial_division[:100]


[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541]

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541]